In [12]:
# Basic decorator (no arguments)

def logger(func):
    def wrapper(*args, **kwargs):
        print(f"calling {func.__name__} with {args} {kwargs}")
        result = func(*args, **kwargs)
        print(f"Result: {result}")
        return result
    return wrapper

@logger
def add(a, b):
    return a + b

add(3, b=7)

calling add with (3,) {'b': 7}
Result: 10


10

In [13]:
# 2. Decorator with arguments
def repeat(n):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for i in range(n):
                func(*args, **kwargs)
        return wrapper
    return decorator

@repeat(3)
def hello():
    print("Hello!")

hello()

Hello!
Hello!
Hello!


In [14]:
# 3. Decorator preserving function metadata (functools.wraps)

from functools import wraps

def uppercase(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result.upper()
    return wrapper

@uppercase
def greet(name):
    return f"hello, {name}"

print(greet("Mohan"))
print(greet.__name__)
print(greet.__doc__)

HELLO, MOHAN
greet
None


In [15]:
# 4. Multiple decorators applied

def exlaim(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs) + "!"
    return wrapper

def question(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs) + "?"
    return wrapper

@exlaim
@question
def greet():
    return "Hello"

print(greet())

Hello?!


In [16]:
# 5. Decorator with both positional and keyword arguments

def debug(func):
    def wrapper(*args, **kwargs):
        print(f"Args: {args}, kwargs: {kwargs}")
        return func(*args, **kwargs)
    return wrapper

@debug
def divide(a, b=1):
    return a/b

print(divide(10, b=2))

Args: (10,), kwargs: {'b': 2}
5.0


In [17]:
# 1. Basic Class-based Decorator

class logger:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        print(f"starting '{self.func.__name__}'...")
        result = self.func(*args, **kwargs)
        print(f"Finished '{self.func.__name__}'.")
        return result
    
@logger
def greet(name):
    print(f"Hello, {name}")

greet("Mohan")

starting 'greet'...
Hello, Mohan
Finished 'greet'.


In [18]:
# 2. Class-based Decorator with Arguments

class prefixlogger:
    def __init__(self, prefix):
        self.prefix = prefix

    def __call__(self, func):
        def wrapper(*args, **kwargs):
            print(f"{self.prefix} - Executing {func.__name__}")
            return func(*args, **kwargs)
        return wrapper
    
@prefixlogger("DEBUG")
def add(a, b):
    return a + b

print(add(5, 7))

DEBUG - Executing add
12


In [19]:
# 3. Caching with Class-based Decorator

class cache:
    def __init__(self, func):
        self.func = func
        self.cache = {}

    def __call__(self, *args):
        if args in self.cache:
            print("Fetching from cache...")
            return self.cache[args]
        print("Computing result...")
        result = self.func(*args)
        self.cache[args] = result
        return result
    
@cache
def multiply(a, b):
    return a * b

print(multiply(3, 4))
print(multiply(3, 4))

Computing result...
12
Fetching from cache...
12


In [20]:
# 4. Retry Logic Decorator

import random

class retry:
    def __init__(self, retries):
        self.retries  =  retries

    def __call__(self, func):
        def wrapper(*args, **kwargs):
            for attempt in range(1, self.retries + 1):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    print(f"Attempt {attempt} failed: {e}")
            raise RuntimeError("All retries failed")
        return wrapper


@retry(3)
def unstabletask():
    if random.random() < 0.7:
        raise ValueError("Random failure")
    return "Success!"

print(unstabletask())

Success!


In [21]:
# 5. Timing Function Execution

import time

class timer:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        start = time.time()
        result = self.func(*args, **kwargs)
        end = time.time()
        print(f"{self.func.__name__} took {end - start:.4f} seconds")
        return result
    
@timer
def slowfunction():
    time.sleep(1)

    return "Done!"

slowfunction()

slowfunction took 1.0082 seconds


'Done!'

In [22]:
# 6. Argument Validation

class postiveargs:
    def __init__(self, func):
        self.func = func
    
    def __call__(self, *args):
        if not all(isinstance(a, int) and a > 0 for a in args):
            raise ValueError("All arguments must be positive integers")
        return self.func(*args)
    


@postiveargs
def add(a, b):
    return a + b

print(add(3, 5))

8


In [23]:
# 7. Access Control Decorator

class requireadmin:
    def __init__(self, func):
        self.func = func

    def __call__(self, userrole, *args, **kwargs):
        if userrole != "admin":
            raise PermissionError("User must be an admin")
        return self.func(userrole, *args, **kwargs)
    
@requireadmin
def deleteuser(role, username):
    print(f"User {username} deleted.")

deleteuser("admin", "John doe")
deleteuser("user", "John doe")

User John doe deleted.


PermissionError: User must be an admin

In [ ]:
# 8. Class-based Decorator with State Tracking

class callcounter:
    def __init__(self, func):
        self.func = func
        self.count = 0

    def __call__(self, *args, **kwargs):
        self.count += 1
        print(f"{self.func.__name__} has been called {self.count} times")
        return self.func(*args, **kwargs)
    
@callcounter
def greet(name):
    print(f"Hello, {name}")

greet("Mohan")
greet("Balor")

greet has been called 1 times
Hello, Mohan
greet has been called 2 times
Hello, Balor


In [ ]:
# 9. Parameterized Class Decorator for Rate Limiting

import time

class ratelimit:
    def __init__(self, seconds):
        self.seconds = seconds
        self.lastcalled = 0
        
    def __call__(self, func):
        def wrapper(*args, **kwargs):
            now = time.time()
            if now - self.lastcalled < self.seconds:
                raise RuntimeError("Too many calls")
            self.lastcalled = now
            return func(*args, **kwargs)
        return wrapper
    
@ratelimit(2)
def ping():
    print("Ping!")

ping()


Ping!


In [ ]:
# 10. Stacking Multiple Class-based Decorators

class uppercase:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs).upper()
    
class exclaim:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs) + "!!!"
    
@exclaim
@uppercase
def greet(name):
    return f"Hello, {name}"

greet("alice")

'HELLO, ALICE!!!'

In [ ]:
# Basic Definition: Write a function greet() that returns "hello". Apply the @uppercase decorator to make its output uppercase.

def uppercase(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result.upper() if isinstance(result, str) else result 
    return wrapper

@uppercase
def greet():
    return "hello"

print(greet())

HELLO


In [ ]:
# Decorator Chaining: Apply both @uppercase and @exclaim to the greet() function. Try both orders:

def uppercase(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result.upper() if isinstance(result, str) else result
    return wrapper

def exlaim(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result + "!!!" if isinstance(result, str) else result
    return wrapper

@uppercase
@exlaim
def greet():
    return "hello"

print(greet())

@exlaim
@uppercase
def greet():
    return "hello"

print(greet())

HELLO!!!
HELLO!!!


In [3]:
# Basic Definition: Write a @reverse decorator that reverses the string returned by a function greet() which outputs "hello".

def reverse(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result[::-1] if isinstance(result, str) else result
    return wrapper

@reverse
def greet():
    return "hello"

print(greet())

olleh


In [5]:
def reverse(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result[::-1] if isinstance(result, str) else result
    return wrapper

def starborder(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return f"***{result}***" if isinstance(result, str) else result
    return wrapper

@starborder
@reverse
def greet():
    return "hello"

@reverse
@starborder
def greet2():
    return "hello"

print(greet())
print(greet2())

***olleh***
***olleh***


In [ ]:
def first(func):
    def wrapper(*args, **kwargs):
        print("first")
        return func(*args, **kwargs)
    return wrapper

def second(func):
    def wrapper(*args, **kwargs):
        print("second")
        return func(*args, **kwargs)
    return wrapper

@first
@second
def greet():
    print("hello")

greet()

print('\n')

@second
@first
def greet1():
    print("Hello")

greet1()

first
second
hello


second
first
Hello


In [13]:
def double(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs) * 2
    return wrapper

def square(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs) ** 2
    return wrapper

def negate(func):
    def wrapper(*args, **kwargs):
        return -func(*args, **kwargs)
    return wrapper

def getnumber():
    return 3

@negate
@square
@double
def num1():
    return 3

@negate
@double
@square
def num2():
    return 4

@square
@double
@negate
def num3():
    return 2

print(num1())
print(num2())
print(num3())

-36
-32
16


In [15]:
def logargs(func):
    def wrapper(*args, **kwargs):
        print(f"Argument: args={args}, kwargs={kwargs}")
        return func(*args, **kwargs)
    return wrapper

def logresult(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f"Result: {result}")
        return result
    return wrapper

@logresult
@logargs
def add(a, b):
    return a + b

add(5,b=7)

@logargs
@logresult
def multiply(a, b):
    return a * b

multiply(3, b=4)

Argument: args=(5,), kwargs={'b': 7}
Result: 12
Argument: args=(3,), kwargs={'b': 4}
Result: 12


12

In [23]:
import time
from functools import wraps

def timings(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"Execution time for {func.__name__}({args}): {end -start:.6f}s")
        return result
    return wrapper

def cache(func):
    memo = {}
    @wraps(func)
    def wrapper(n):
        if n in memo:
            return memo[n]
        result = func(n)
        memo[n] = result
        return result
    return wrapper

@timings
@cache
def fib1(n):
    if n <= 1:
        return n
    return fib1(n-1) + fib1(n - 2)

fib1(10)
print("\n")

@cache
@timings
def fib2(n):
    if n<= 1:
        return n
    return fib2(n-1) + fib2(n-2)

fib2(8)

Execution time for fib1((1,)): 0.000000s
Execution time for fib1((0,)): 0.000000s
Execution time for fib1((2,)): 0.000000s
Execution time for fib1((1,)): 0.000000s
Execution time for fib1((3,)): 0.000000s
Execution time for fib1((2,)): 0.000000s
Execution time for fib1((4,)): 0.000000s
Execution time for fib1((3,)): 0.000000s
Execution time for fib1((5,)): 0.000000s
Execution time for fib1((4,)): 0.000000s
Execution time for fib1((6,)): 0.000000s
Execution time for fib1((5,)): 0.000000s
Execution time for fib1((7,)): 0.000000s
Execution time for fib1((6,)): 0.000000s
Execution time for fib1((8,)): 0.000000s
Execution time for fib1((7,)): 0.000000s
Execution time for fib1((9,)): 0.000000s
Execution time for fib1((8,)): 0.000000s
Execution time for fib1((10,)): 0.000000s


Execution time for fib2((1,)): 0.000000s
Execution time for fib2((0,)): 0.000000s
Execution time for fib2((2,)): 0.000000s
Execution time for fib2((3,)): 0.000000s
Execution time for fib2((4,)): 0.000000s
Execution tim

21

In [28]:
import time
from functools import wraps

userloggedin = True

def requirelogin(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        if not userloggedin:
            print("Access denied: User not logged in!")
            return None
        return func(*args, **kwargs)
    return wrapper

def logcall(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(f"Function '{func.__name__}' called at {time.strftime('%Y-%m-%d %H:%M:%S')}")
        return func(*args, **kwargs)
    return wrapper

@logcall
@requirelogin
def getdata1():
    print("Fetching senstive data...")
    return {"data": [1,2,3]}

@requirelogin
@logcall
def getdata2():
    print("Fetching sensitive data...")
    return {"data": [1,2,3]}



print(getdata1())
print('\n')

print(getdata2())

Function 'getdata1' called at 2025-08-18 20:27:34
Fetching senstive data...
{'data': [1, 2, 3]}


Function 'getdata2' called at 2025-08-18 20:27:34
Fetching sensitive data...
{'data': [1, 2, 3]}


In [ ]:
import time
import random
import signal
from functools import wraps

def retry(n):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for attempt in range(1, n + 1):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    print(f"Attempt {attempt} failed: {e}")
            raise RuntimeError(f"Function failed after {n} retries")
        return wrapper
    return decorator

def timeout(t):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            def handler(signum, frame):
                raise TimeoutError(f"function timed out after {t} seconds")
            signal.signal(signal.SIGALRM, handler)
            signal.alarm(t)
            try:
                return func(*args, **kwargs)
            finally:
                signal.alarm(0)
        return wrapper
    return decorator
    
def unstablefunction():
    delay = random.uniform(0, 3)
    print(f"Running unstable function (delay={delay:.2f}s)...")
    time.sleep(delay)
    if random.random() < 0.5:
        raise ValueError("Random failure!")
    return "Success!"

@timeout(2)
@retry(5)
def test1():
    return unstablefunction()

@retry(5)
@timeout(2)
def test2():
    return unstablefunction()

try:
    print("Result: ", test1())
except Exception as e:
    print("Final Exception: ", e)

print('\n')



Final Exception:  module 'signal' has no attribute 'SIGALRM'


Attempt 1 failed: module 'signal' has no attribute 'SIGALRM'
Attempt 2 failed: module 'signal' has no attribute 'SIGALRM'
Attempt 3 failed: module 'signal' has no attribute 'SIGALRM'
Attempt 4 failed: module 'signal' has no attribute 'SIGALRM'
Attempt 5 failed: module 'signal' has no attribute 'SIGALRM'
Final Exception:  Function failed after 5 retries


In [32]:
try:
    print("Result: ", test2())
except Exception as e:
    print("Final Exception: ", e)

Attempt 1 failed: module 'signal' has no attribute 'SIGALRM'
Attempt 2 failed: module 'signal' has no attribute 'SIGALRM'
Attempt 3 failed: module 'signal' has no attribute 'SIGALRM'
Attempt 4 failed: module 'signal' has no attribute 'SIGALRM'
Attempt 5 failed: module 'signal' has no attribute 'SIGALRM'
Final Exception:  Function failed after 5 retries


In [35]:
from functools import wraps

def countcall(decoratorfunc):
    @wraps(decoratorfunc)
    def wrapper(*dargs, **dkwargs):
        decoratedfunc = decoratorfunc(*dargs, **dkwargs)
        decoratedfunc.call_count = 0

        @wraps(decoratedfunc)
        def inner(*args, **kwargs):
            decoratedfunc.call_count += 1
            print(f"[{decoratedfunc.__name__}] call count: {decoratedfunc.call_count}")
            return decoratedfunc(*args, **kwargs)
        return inner
    return wrapper

def uppercase(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result.upper() if isinstance(result, str) else result
    return wrapper

@countcall
def greetdecorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

@greetdecorator
@uppercase
def greet(name):
    return f"hello {name}"

greet("Monan")
greet("Balor")
greet("Finn")

[greet] call count: 1
[greet] call count: 2
[greet] call count: 3


'HELLO FINN'